In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv




In [2]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 44.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is inc

In [3]:
#Install libraries
!pip install utils
!pip install python-utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=6a454a211d7ec9971234e5fa3f503e460e3f9a7132c66c053d480d6999ec3c16
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [4]:
# Шлях до досліджуваного відофайлу
PATH_TO_VIDEOFILE = "/kaggle/input/deepfake-detection-challenge/train_sample_videos/bilnggbxgu.mp4"

In [5]:
# ПІДРАХУНОК КІЛЬКОСТІ КЛІПАНЬ



import mediapipe as mp
import time
import utils, math
import numpy as np
# variables 
frame_counter =0
CEF_COUNTER =0
SIMPLE_BLINK = 0
TOTAL_BLINKS =0
DECISION_STATUS = 0
# constants
CLOSED_EYES_FRAME =3
FONTS =cv.FONT_HERSHEY_COMPLEX

# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]

# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

map_face_mesh = mp.solutions.face_mesh

camera = cv.VideoCapture(PATH_TO_VIDEOFILE)

def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks 
    return mesh_coord

# Euclaidean distance 
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance

# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio 


with map_face_mesh.FaceMesh(min_detection_confidence =0.5, min_tracking_confidence=0.5) as face_mesh:

    # starting time here 
    start_time = time.time()
    # starting Video loop here.
    while True:
        frame_counter +=1 # frame counter
        ret, frame = camera.read() # getting frame from camera 
        if not ret: 
            break # no more frames break
        #  resizing frame
        
        frame = cv.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv.INTER_CUBIC)
        frame_height, frame_width= frame.shape[:2]
        rgb_frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        results  = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_coords = landmarksDetection(frame, results, False)
            ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
            
            if ratio >5.5:
                CEF_COUNTER +=1
                SIMPLE_BLINK +=1
            else:
                if CEF_COUNTER>CLOSED_EYES_FRAME:
                    TOTAL_BLINKS +=1
                    CEF_COUNTER =0

        # calculating  frame per seconds FPS
        end_time = time.time()-start_time
        fps = frame_counter/end_time

    # змінна, що відповідає за обчислення обтимальної кількості кліпань
    NORMAL_BLINK_COUNT = 0    
    # умова кількості кліпань (в середньому відбувається одне кліпання на 5 секунд)
    if end_time < 5:
        #якщо відео коротке, то вважаємо що 1 кліпання достатньо, щоби вважати відео оригінальним
        NORMAL_BLINK_COUNT = 1
    else:
        # якщо відео довше 5 секунд, то підраховувємо співвідношення (к-сть секунд відео / середній інтервал кліпань
        NORMAL_BLINK_COUNT = end_time/5
    
    if(TOTAL_BLINKS < NORMAL_BLINK_COUNT):
        # Імовірність діпфейку - середня 
        DECISION_STATUS = "Імовірність діпфейку - середня"
    else:
        # Імовірність діпфейку - низька
        DECISION_STATUS = "Імовірність діпфейку - низька"

    print("Тривалість відео: " + str(end_time))
    print("simple blink: " + str(SIMPLE_BLINK))
    print("Кількість кліпань: " + str(TOTAL_BLINKS))
    print("Рішення " + DECISION_STATUS)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1728884457.072493     128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728884457.091051     128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728884457.166712     127 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/conda/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Тривалість відео: 6.959192752838135
simple blink: 40
Кількість кліпань: 7
Рішення Імовірність діпфейку - низька


In [6]:
!pip install '/kaggle/input/dlibpkg/dlib-19.19.0'
!pip install '/kaggle/input/face-recognition-models-0-3-0-hdfd/kaggle/input/face-recognition/face_recognition_models-0.3.0/face_recognition_models-0.3.0'
!pip install '/kaggle/input/face-recognition-0-1-5-py2-py3-none-any/face_recognition-0.1.5-py2.py3-none-any.whl'
!pip install '/kaggle/input/imageio-ffmpeg1/imageio_ffmpeg-0.3.0-py3-none-manylinux2010_x86_64.whl'


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Processing /kaggle/input/dlibpkg/dlib-19.19.0
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
      error: [Errno 30] Read-only file system: '/kaggle/input/dlibpkg/dlib-19.19.0/build'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
  Running setup.py clean for dlib
Failed to build dlib
ERROR: Could not build wheels for dlib, which is required to install pyproject.toml-based projects
Processing /kaggle/input/face-recognition-models-0-3-0-hdfd/kaggle/input/face-recognition/face_recognition_models-0.3.0/face_recognition_models-0.3.0
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run succ

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import glob
import cv2
from albumentations import *
from tqdm import tqdm_notebook as tqdm
import gc

from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from keras.optimizers import Adam
import face_recognition
import imageio
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')
PATH = '../input/deepfake-detection-challenge/'
print(os.listdir(PATH))

for dirname, _, filenames in os.walk('/kaggle/input/meso-pretrain'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from IPython.display import HTML
from base64 import b64encode
vid1 = open('/kaggle/input/deepfake-detection-challenge/test_videos/ytddugrwph.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(vid1).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)




class Video:
    def __init__(self, path):
        self.path = path
        self.container = imageio.get_reader(path, 'ffmpeg')
        self.length = self.container.count_frames()
#         self.length = self.container.get_meta_data()['nframes']
        self.fps = self.container.get_meta_data()['fps']
    
    def init_head(self):
        self.container.set_image_index(0)
    
    def next_frame(self):
        self.container.get_next_data()
    
    def get(self, key):
        return self.container.get_data(key)
    
    def __call__(self, key):
        return self.get(key)
    
    def __len__(self):
        return self.length
    
    

    
    
    
    
    
    
IMGWIDTH = 256

class Classifier:
    def __init__():
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)


class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def init_model(self): 
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))
        
        x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

class MesoInception4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(learning_rate = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    
    def InceptionLayer(self, a, b, c, d):
        def func(x):
            x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
            
            x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
            x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
            
            x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
            x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='relu')(x3)
            
            x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
            x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='relu')(x4)

            y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
            return y
        return func
    
    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))
        
        x1 = self.InceptionLayer(1, 4, 4, 2)(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = self.InceptionLayer(2, 4, 4, 2)(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)
    
    
    
    
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)



classifier = MesoInception4()
classifier.load('/kaggle/input/mesonetinch5/MesoInception_DF.h5')

# 0 fake
# 1 real 


submit = []



save_interval = 1 # perform face detection every {save_interval} frames
margin = 0.2
# for vi in os.listdir('/kaggle/input/deepfake-detection-challenge/test_videos'):
#     print(os.path.join("/kaggle/input/deepfake-detection-challenge/test_videos/", vi))
re_video = 0.5
try:
    video1 = Video('/kaggle/input/deepfake-detection-challenge/train_sample_videos/aagfhgtpmv.mp4')
    re_imgs = []
    for i in range(0,video1.__len__(),save_interval):
        img = video1.get(i)
        face_positions = face_recognition.face_locations(img)
        for face_position in face_positions:
            offset = round(margin * (face_position[2] - face_position[0]))
            y0 = max(face_position[0] - offset, 0)
            x1 = min(face_position[1] + offset, img.shape[1])
            y1 = min(face_position[2] + offset, img.shape[0])
            x0 = max(face_position[3] - offset, 0)
            face = img[y0:y1,x0:x1]

            inp = cv2.resize(face,(256,256))/255.
            re_img = classifier.predict(np.array([inp]))
            print(video1,": ",i , "  :  ",classifier.predict(np.array([inp])))
            re_imgs.append(re_img[0][0])
    re_video = np.average(re_imgs)
    print("re_video = " + str(re_video))
    if np.isnan(re_video):
        re_video = 0.5
except:
    re_video = 0.5
    print("re_video = " + str(re_video))
    submit.append([video1,1.0-re_video])
#     submit.append([vi,re_video])

#     submit[vi] = 1.0-re_video
#     print(vi,": ",str(1.0-re_video))

['test_videos', 'sample_submission.csv', 'train_sample_videos']
/kaggle/input/meso-pretrain/MesoInception_F2F
/kaggle/input/meso-pretrain/MesoInception_DF
/kaggle/input/meso-pretrain/mesonet4_kaggle.h5
/kaggle/input/meso-pretrain/Meso4_F2F
/kaggle/input/meso-pretrain/Meso4_DF
/kaggle/input/meso-pretrain/mesonet41.h5


I0000 00:00:1728885766.481084    1542 service.cc:145] XLA service 0x7c41f4007fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728885766.481141    1542 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728885766.481145    1542 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


I0000 00:00:1728885768.589019    1542 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


<__main__.Video object at 0x7c429f3bda80> :  0   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
<__main__.Video object at 0x7c429f3bda80> :  1   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
<__main__.Video object at 0x7c429f3bda80> :  2   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
<__main__.Video object at 0x7c429f3bda80> :  3   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
<__main__.Video object at 0x7c429f3bda80> :  4   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
<__main__.Video object at 0x7c429f3bda80> :  5   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
<__main__.Video object at 0x7c429f3bda80> :  6   :   [[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
<__main__.Video object at 0x7c429f3bda80> :  7  

In [ ]:
'''
# Для документації - ПІДРАХУНОК КІЛЬКОСТІ КЛІПАНЬ

#
# Функція розроблена для обчислення кількості
# кліпань особою на відео
# Процес обчислення полягає у визначені 
# Евклідової відстані між точками навколо очей
# Розроблено механізм підрахунку та визнаяення "фейк / не фейк"
def calculateResult(videoFile):
    
    # змінна для обчислення тривалості відео потоку
    start_time = time.time()
    while True:
        # обчислення кількості фреймів на відео для їх подальшого окремого оброблення 
        frame_counter +=1 
        
        # зберігання поточного фрейму у змінну, джерелом фрейму є 
        frame = videoFile.read() 
        
        # зміна розмірів зображення
        frame = cv.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv.INTER_CUBIC)
        frame_height, frame_width= frame.shape[:2]
        
        # перетворення формату зображення
        rgb_frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        results  = face_mesh.process(rgb_frame)
        
        # пошук ключових точок на зображенні для подальшого обчислення
        # відношення між точками очей
        # та визначення - відкритості - закритості очей
        if results.multi_face_landmarks:
            mesh_coords = landmarksDetection(frame, results, False)
            ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
            
            if ratio >5.5:
                CEF_COUNTER +=1
                SIMPLE_BLINK +=1
            else:
                if CEF_COUNTER>CLOSED_EYES_FRAME:
                    TOTAL_BLINKS +=1
                    CEF_COUNTER =0

        # calculating  frame per seconds FPS
        end_time = time.time()-start_time
        fps = frame_counter/end_time

    # змінна, що відповідає за обчислення обтимальної кількості кліпань
    NORMAL_BLINK_COUNT = 0    
    # умова кількості кліпань (в середньому відбувається одне кліпання на 5 секунд)
    if end_time < 5:
        #якщо відео коротке, то вважаємо що 1 кліпання достатньо, щоби вважати відео оригінальним
        NORMAL_BLINK_COUNT = 1
    else:
        # якщо відео довше 5 секунд, то підраховувємо співвідношення (к-сть секунд відео / середній інтервал кліпань
        NORMAL_BLINK_COUNT = end_time/5
    
    if(TOTAL_BLINKS < NORMAL_BLINK_COUNT):
        # Імовірність діпфейку - середня 
        DECISION_STATUS = 1
    else:
        # Імовірність діпфейку - низька
        DECISION_STATUS = 2

    print("Тривалість відео: " + str(end_time))
    print("simple blink: " + str(SIMPLE_BLINK))
    print("Кількість кліпань: " + str(TOTAL_BLINKS))
    print("Рішення " + str(DECISION_STATUS))

'''

In [ ]:
'''
# Для документації - АРХІТЕКТУРА ЗГОРТКОВОЇ НЕЙРОННОЇ МЕРЕЖІ

# Функція опису архітектури згорткової нейронної мережі
# для класифікації діпфейків
# на вхід подається зображення розміром 256 x 256 пікселів
def define_model(shape=(256,256,3)):
    inputImage = Input(shape = shape)
    
    # перший блок
    firstBlock_1 = InceptionLayer(1, 4, 4, 2)(inputImage)
    firstBlock_2 = BatchNormalization()(firstBlock_1)
    firstBlock_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(firstBlock_2)
    
    # другий блок
    secondBlock_1 = InceptionLayer(2, 4, 4, 2)(firstBlock_3)
    secondBlock_2 = BatchNormalization()(secondBlock_1)        
    secondBlock_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(secondBlock_2)   
    
    # третій блок    
    thirdBlock_1 = Conv2D(16, (3, 3), padding='same', activation = 'elu')(secondBlock_3)
    thirdBlock_2 = BatchNormalization()(thirdBlock_1)
    thirdBlock_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(thirdBlock_2)
    
    # четвертий блок
    fourthBlock_1 = Conv2D(16, (3, 3), padding='same', activation = 'elu')(thirdBlock_3)
    fourthBlock_2 = BatchNormalization()(fourthBlock_1)
    fourthBlock_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(fourthBlock_2)

    # п'ятий блок
    fifthBlock_1 = Conv2D(32, (5, 5), padding='same', activation = 'elu')(fourthBlock_3)
    fifthBlock_2 = BatchNormalization()(fifthBlock_1)
    fifthBlock_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(fifthBlock_2)
    
    final = Flatten()(fifthBlock_3)
    final = Dropout(0.5)(final)
    final = Dense(16)(final)
    final = LeakyReLU(alpha=0.1)(final)
    final = Dropout(0.5)(final)
    final = Dense(1, activation = 'sigmoid')(final)
    
    model=Model(inputs = x, outputs = y)
    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4))
    return model
'''